In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from math import nan
import glob
import dask
import datetime
import pandas as pd
import sys

from CASutils import filter_utils as filt
from CASutils import readdata_utils as read
dask.config.set(**{'array.slicing.split_large_chunks': True})

In [2]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '10GB',
    processes = 1,
    queue = 'casper',
    local_directory='$TMPDIR',
    resource_spec='select=1:ncpus=1:mem=10GB',
    project='P04010022',
    walltime='05:00:00',
    interface='ext')

# scale up
cluster.scale(12)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

/glade/u/home/islas/miniconda3/envs/islaenv24/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44245 instead
  f"Port {expected} is already in use.\n"


In [5]:
cluster

PBSCluster(d2961d03, 'tcp://128.117.208.81:39403', workers=6, threads=6, memory=55.86 GiB)

In [6]:
client

<Client: 'tcp://128.117.208.81:39403' processes=6 threads=6, memory=55.86 GiB>

### Read in the data

In [7]:
ystart=1850 ; yend=2099

pathout="/glade/campaign/cgd/cas/islas/python/L83/DATA_SORT/MJO_OMEGA500/"

In [8]:
for imem in np.arange(1,3+1,1):
    basepathhist="/glade/campaign/cesm/development/cvcwg/cvwg/L83/timeseries/b.e21.BHISTcmip6.f09_g17.L83_cam6."+str(imem).zfill(3)+"/atm/proc/tseries/day_1/"
    uzmhist = xr.open_mfdataset(basepathhist+"*.Uzm.*.nc")
    uzmhist = read.fixcesmtime_daily(uzmhist)
    uzmhist = uzmhist.where( uzmhist.time.dt.hour == 12, drop=True)
    omegahist = xr.open_mfdataset(basepathhist+"*.OMEGA500.*.nc")
    omegahist = read.fixcesmtime_daily(omegahist)
    omegahist = omegahist.where( omegahist.time.dt.hour == 12, drop=True)
    omegahist['lon'] = uzmhist.lon ; omegahist['lat'] = uzmhist.lat

    basepathssp = "/glade/campaign/cesm/development/cvcwg/cvwg/L83/timeseries/b.e21.BSSP370cmip6.f09_g17.L83_cam6."+str(imem).zfill(3)+"/atm/proc/tseries/day_1/"
    uzmssp = xr.open_mfdataset(basepathssp+"*.Uzm.*.nc")
    uzmssp = read.fixcesmtime_daily(uzmssp)
    uzmssp['lon'] = uzmhist.lon ; uzmssp['lat'] = uzmhist.lat
    omegassp = xr.open_mfdataset(basepathssp+"*.OMEGA500.*.nc")
    omegassp = read.fixcesmtime_daily(omegassp)
    omegassp = omegassp.where( omegassp.time.dt.hour == 12, drop=True)
    omegassp['lon'] = uzmhist.lon ; omegassp['lat'] = uzmhist.lat

    uzm = xr.concat([uzmhist, uzmssp], dim='time')
    omega = xr.concat([omegahist, omegassp], dim='time')

    uzm = uzm.sel(time=slice(str(ystart)+"-01-01",str(yend+1)+"-12-31"))
    uzm = uzm.Uzm.isel(zlon=0)

    omega = omega.sel(time=slice(str(ystart)+"-01-01",str(yend+1)+"-12-31"))
    omega = omega.OMEGA500

    alluzm=[]
    allomega=[]
    for iyear in np.arange(ystart,yend+1,1):
        print(iyear)
        uzmuse = uzm.sel(time=slice(str(iyear)+"-12-01",str(iyear+1)+"-02-28")).mean('time')
        alluzm.append(uzmuse)

        omegause = omega.sel(time=slice(str(iyear)+"-11-01",str(iyear+1)+"-03-31"))
        omegaclim = omegause.mean('time')
        omegaanoms = omegause - omegaclim
        omegaanoms['time'] = pd.date_range("1970-11-01","1971-03-31")
        mjoomega = filt.wkfilter(omegaanoms, 0.15, 1, 5, 20, 100, spd=1)
        mjoomega = mjoomega.sel(time=slice("1970-12-01","1971-02-28"))
        allomega.append(mjoomega)

    year = xr.DataArray(np.arange(ystart+1,yend+2,1), coords=[np.arange(ystart+1, yend+2, 1)], dims=['year'], name='year')
    allomega = xr.concat(allomega, dim=year)
    alluzm = xr.concat(alluzm, dim=year)
    allomega = allomega.rename('MJO_OMEGA500')
    alluzm = alluzm.rename('Uzm')

    ds = xr.merge([allomega, alluzm])
    ds.to_netcdf(pathout+'MJO_OMEGA500_BL83_'+str(ystart)+'_'+str(yend+1)+'_'+str(imem).zfill(3)+'.nc')

cluster.close()

1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049


distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
ERROR:asyncio:_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


In [13]:
omega

<xarray.DataArray 'OMEGA500' (time: 15331, lat: 192, lon: 288)>
dask.array<getitem, shape=(15331, 192, 288), dtype=float32, chunksize=(3650, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) datetime64[ns] 1979-01-01T12:00:00 ... 2020-12-31T12:00:00
Attributes:
    units:         Pa/s
    long_name:     Vertical velocity at 500 mbar pressure surface
    cell_methods:  time: mean

In [18]:
for itime in np.arange(0,omegause.time.size,1):
    print(omegause.time.isel(time=itime).values)

2014-11-01T12:00:00.000000000
2014-11-02T12:00:00.000000000
2014-11-03T12:00:00.000000000
2014-11-04T12:00:00.000000000
2014-11-05T12:00:00.000000000
2014-11-06T12:00:00.000000000
2014-11-07T12:00:00.000000000
2014-11-08T12:00:00.000000000
2014-11-09T12:00:00.000000000
2014-11-10T12:00:00.000000000
2014-11-11T12:00:00.000000000
2014-11-12T12:00:00.000000000
2014-11-13T12:00:00.000000000
2014-11-14T12:00:00.000000000
2014-11-15T12:00:00.000000000
2014-11-16T12:00:00.000000000
2014-11-17T12:00:00.000000000
2014-11-18T12:00:00.000000000
2014-11-19T12:00:00.000000000
2014-11-20T12:00:00.000000000
2014-11-21T12:00:00.000000000
2014-11-22T12:00:00.000000000
2014-11-23T12:00:00.000000000
2014-11-24T12:00:00.000000000
2014-11-25T12:00:00.000000000
2014-11-26T12:00:00.000000000
2014-11-27T12:00:00.000000000
2014-11-28T12:00:00.000000000
2014-11-29T12:00:00.000000000
2014-11-30T12:00:00.000000000
2014-12-01T12:00:00.000000000
2014-12-02T12:00:00.000000000
2014-12-03T12:00:00.000000000
2014-12-04

In [14]:
omegause

<xarray.DataArray 'OMEGA500' (time: 152, lat: 192, lon: 288)>
dask.array<getitem, shape=(152, 192, 288), dtype=float32, chunksize=(91, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) datetime64[ns] 2014-11-01T12:00:00 ... 2015-03-31T12:00:00
Attributes:
    units:         Pa/s
    long_name:     Vertical velocity at 500 mbar pressure surface
    cell_methods:  time: mean

In [17]:
print(basepathssp)

/glade/campaign/cesm/development/cvcwg/L83/timeseries/f.e21.FHIST_BGC.f09_f09_mg17.L83_cam6_SSP370.001/atm/proc/tseries/day_1/
